In [ ]:
from pathlib import Path
import datetime
from zoneinfo import ZoneInfo
import shutil
from neuroconv.utils import load_dict_from_file, dict_deep_update
import h5py
import numpy as np
from pynwb import NWBFile

In [ ]:
from manimoh_nwb_converters import OdorSeqNWBConverter


In [ ]:
def session_to_nwb(data_dir_path: str | Path, output_dir_path: str | Path, stub_test: bool = False):

    data_dir_path = Path(data_dir_path)
    output_dir_path = Path(output_dir_path)
    if stub_test:
        output_dir_path = output_dir_path / "nwb_stub"
    output_dir_path.mkdir(parents=True, exist_ok=True)

    session_id = "sample_session"
    subject_id = "SL18"
    nwbfile_path = output_dir_path / f"{session_id}.nwb"

    source_data = dict()
    conversion_options = dict()

    # Add Ephys (can we do without this?)
    # file_path = (
    #     data_dir_path
    #     / "SL18_D19"
    #     / "SL18_D19_S01_F01_BOX_SLP_20230503_112642"
    #     / "SL18_D19_S01_F01_BOX_SLP_20230503_112642.rec"
    # )
    # source_data.update(dict(Recording=dict(file_path=file_path)))
    # conversion_options.update(dict(Recording=dict(stub_test=stub_test)))

    # # Add Sorting
    # spike_times_folder_path = data_dir_path / "SL18_D19" / "SL18_D19.SpikesFinal"
    # unit_stats_folder_path = data_dir_path / "SL18_D19" / "SL18_D19.ExportedUnitStats"
    # source_data.update(
    #     dict(
    #         Sorting=dict(spike_times_folder_path=spike_times_folder_path, unit_stats_folder_path=unit_stats_folder_path)
    #     )
    # )
    # conversion_options.update(dict(Sorting=dict()))

    # Add LFP
    folder_path = data_dir_path / "SL18_D19" / "SL18_D19.LFP"
    source_data.update(dict(LFP=dict(folder_path=folder_path)))
    conversion_options.update(dict(LFP=dict(stub_test=stub_test)))


    # # Add Behavior
    # folder_path = "/Volumes/T7/CatalystNeuro/Jadhav/SubLearnProject/SL18_D19/SL18_D19.DIO"
    # source_data.update(dict(Behavior=dict(folder_path=folder_path)))
    # conversion_options.update(dict(Behavior=dict()))

    converter = OdorSeqNWBConverter(source_data=source_data)

    # Add datetime to conversion
    metadata = converter.get_metadata()
    metadata["NWBFile"]["session_start_time"] = datetime.datetime(2023, 5, 3, 11, 26, 42, tzinfo=ZoneInfo("US/Eastern"))

    # Update default metadata with the editable in the corresponding yaml file
    editable_metadata_path = Path(__file__).parent / "olson_2024_metadata.yaml"
    editable_metadata = load_dict_from_file(editable_metadata_path)
    metadata = dict_deep_update(metadata, editable_metadata)

    # Run conversion
    converter.run_conversion(metadata=metadata, nwbfile_path=nwbfile_path, conversion_options=conversion_options)

In [17]:
file = h5py.File('E:\\odor-pixels\\M541-2024-08-31\\imec0_clean_lfp.mat', 'r') 

In [90]:
file['imec0'].keys()

<KeysViewHDF5 ['channel_ids', 'depths', 'lfp_fs', 'lfp_traces', 'lfp_tvec', 'shank_ids']>

In [142]:
# Code to get channel ids in array
q = np.asarray(file['imec0']['channel_ids'][:], dtype='uint32')
q = q.T.view('U1')
channel_ids = np.asarray([''.join(x).strip() for x in q])
channel_ids

array(['imec0.ap#AP112', 'imec0.ap#AP126', 'imec0.ap#AP138',
       'imec0.ap#AP200', 'imec0.ap#AP214', 'imec0.ap#AP226',
       'imec0.ap#AP288', 'imec0.ap#AP302', 'imec0.ap#AP162',
       'imec0.ap#AP176', 'imec0.ap#AP188', 'imec0.ap#AP250',
       'imec0.ap#AP264', 'imec0.ap#AP276', 'imec0.ap#AP338',
       'imec0.ap#AP352', 'imec0.ap#AP304', 'imec0.ap#AP318',
       'imec0.ap#AP330', 'imec0.ap#AP8', 'imec0.ap#AP22', 'imec0.ap#AP34',
       'imec0.ap#AP96', 'imec0.ap#AP110', 'imec0.ap#AP354',
       'imec0.ap#AP368', 'imec0.ap#AP380', 'imec0.ap#AP58',
       'imec0.ap#AP72', 'imec0.ap#AP84', 'imec0.ap#AP146',
       'imec0.ap#AP160'], dtype='<U14')

In [134]:
# Code to get shank_ids in array
shank_id = file['imec0']['shank_ids'][:].flatten()
shank_id


array([0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2,
       2, 2, 3, 3, 3, 3, 3, 3, 3, 3])

In [110]:
# Code to get depths ids in array TODO: Needs subtraction from ExpKeys.recordingDepth
depths = file['imec0']['depths'][:].flatten()
depths

array([ 480.,  585.,  675.,  780.,  885.,  975., 1080., 1185.,  495.,
        600.,  690.,  795.,  900.,  990., 1095., 1200.,  480.,  585.,
        675.,  780.,  885.,  975., 1080., 1185.,  495.,  600.,  690.,
        795.,  900.,  990., 1095., 1200.])

In [105]:
# Code to get lfp_fs
lfp_fs = file['imec0']['lfp_fs'][:].flatten()
lfp_fs[0]

2500.0

In [116]:
# Code to get lfp_tvec in array
lfp_tvec = file['imec0']['lfp_tvec'][:].flatten()
lfp_tvec

array([0.0000000e+00, 4.0000000e-04, 8.0000000e-04, ..., 6.1901092e+03,
       6.1901096e+03, 6.1901100e+03])

In [ ]:
# Code to get lfp_traces in array
lfp_data= file['imec0']['lfp_traces'][:]
lfp_data

array([[ -30.273438 ,   48.4375   ,  102.92969  , ..., -118.06641  ,
        -105.95703  ,  -93.84766  ],
       [ -48.4375   ,  169.53125  ,  320.89844  , ..., -136.23047  ,
        -112.01172  ,  -81.73828  ],
       [ -33.30078  ,    6.0546875,   30.273438 , ...,  -18.164062 ,
           9.082031 ,   36.328125 ],
       ...,
       [ -18.164062 ,   93.84766  ,  169.53125  , ..., -163.47656  ,
        -148.33984  , -127.14844  ],
       [  -6.0546875,   84.765625 ,  151.36719  , ..., -193.75     ,
        -190.72266  , -181.64062  ],
       [ -27.246094 ,  -27.246094 ,  -27.246094 , ..., -133.20312  ,
        -124.12109  , -112.01172  ]], dtype=float32)

In [130]:
from uuid import uuid4
from datetime import datetime
from dateutil.tz import tzlocal

In [ ]:
nwbfile = NWBFile(
    session_description="my first synthetic recording",
    identifier=str(uuid4()),
    session_start_time=datetime.now(tzlocal()),
    experimenter=[
        "Baggins, Bilbo",
    ],
    lab="Bag End Laboratory",
    institution="University of Middle Earth at the Shire",
    experiment_description="I went on an adventure to reclaim vast treasures.",
    keywords=["ecephys", "exploration", "wanderlust"],
    related_publications="doi:10.1016/j.neuron.2016.12.011",
)
device = nwbfile.create_device(
    name="imec0", description="NPX2.0", manufacturer="IMEC"
)
nwbfile.add_electrode_column(name="channel_id", description="Identifier for the channel on the probe")

unique_shanks = np.unique(shank_id).tolist()
electrode_counter = 0
for iShank in unique_shanks:
    electrode_group = nwbfile.create_electrode_group(
        name="shank{}".format(iShank),
        description="electrode group for shank {}".format(iShank),
        device=device,
        location="brain area", # Need to figure this out 
    )
    # add electrodes to the electrode table
    for ielec,elec in enumerate(np.where(shank_id == iShank)[0]):
        # print
        nwbfile.add_electrode(
            group=electrode_group,
            channel_id = channel_ids[elec],
            location="brain area",  # Need to figure this out 
        )
        electrode_counter += 1
# The line below will display the electrode table
# nwbfile.electrodes.to_dataframe()

In [ ]:
from pynwb.ecephys import ElectricalSeries, LFP
lfp_es = ElectricalSeries(name='LFP', data=lfp_data, electrodes=channel_ids, rate=lfp_fs[0], starting_time=lfp_tvec[0])

TypeError: ElectricalSeries.__init__: incorrect type for 'electrodes' (got 'ndarray', expected 'DynamicTableRegion')